In [1]:
import numpy as np

# Simple Perceptron Learning Algorithm

In [2]:
class SPLA:
  def __init__(self, size, learning_set, answers, learning_rate = 0.1):
    self.size = size
    self.learning_set = learning_set
    self.answers = answers
    self.learning_rate = learning_rate
    self.weights = np.random.randn(size + 1)

  def train(self, iterations):
    for i in range(iterations):
      tmp_i = np.random.randint(0, self.learning_set.shape[0])
      tmp_j = np.random.randint(0, self.learning_set.shape[1])
      predict = self.forward(self.learning_set[tmp_i, tmp_j])
      if self.answers[5*tmp_i + tmp_j] - predict != 0:
        self.weights[1:] += self.learning_rate * (self.answers[5*tmp_i + tmp_j] - predict) * self.learning_set[tmp_i, tmp_j]
        self.weights[0] += self.learning_rate * (self.answers[5*tmp_i + tmp_j] - predict)

  def forward(self, data):
    dot = np.dot(data, self.weights[1:]) + self.weights[0]
    return self.activ_func(dot)

  def activ_func(self, dot):
    return 1 if dot >= 0 else -1

  def predict(self, data):
    return self.forward(data)

# Pocket Learning Algorithm

In [3]:
class PLA:
  def __init__(self, size, learning_set, answers, learning_rate = 0.1):
    self.size = size
    self.learning_set = learning_set
    self.answers = answers
    self.learning_rate = learning_rate
    self.weights = np.random.randn(size + 1)
    self.best_weights = self.weights
    self.best_lifetime = 0
    self.current_lifetime = 0

  def train(self, iterations):
    for i in range(iterations):
      tmp_i = np.random.randint(0, self.learning_set.shape[0])
      tmp_j = np.random.randint(0, self.learning_set.shape[1])
      predict = self.forward(self.learning_set[tmp_i][tmp_j])
      if self.answers[5*tmp_i + tmp_j] - predict != 0:
        self.weights[1:] += self.learning_rate * (self.answers[5*tmp_i + tmp_j] - predict) * self.learning_set[tmp_i][tmp_j]
        self.weights[0] += self.learning_rate * (self.answers[5*tmp_i + tmp_j] - predict)
        self.current_lifetime = 0
      else:
        self.current_lifetime += 1
        if(self.current_lifetime > self.best_lifetime):
          self.best_lifetime = self.current_lifetime
          if (self.best_weights != self.weights).all():
            self.best_weights = np.copy(self.weights)

  def forward(self, data):
    dot = np.dot(data, self.weights[1:]) + self.weights[0]
    return self.activ_func(dot)

  def forward_predict(self, data):
    dot = np.dot(data, self.best_weights[1:]) + self.best_weights[0]
    return self.activ_func(dot)

  def activ_func(self, dot):
    return 1 if dot >= 0 else -1

  def predict(self, data):
    return self.forward_predict(data)

# Pocket Learning Algorithm with Ratchet

In [4]:
class PLAR:
  def __init__(self, size, learning_set, answers, learning_rate = 0.1):
    self.size = size
    self.learning_set = learning_set
    self.answers = answers
    self.learning_rate = learning_rate
    self.weights = np.random.randn(size + 1)
    self.best_weights = self.weights
    self.best_lifetime = 0
    self.best_predict_number = 0
    self.current_lifetime = 0

  def train(self, iterations):
    for i in range(iterations):
      tmp_i = np.random.randint(0, self.learning_set.shape[0])
      tmp_j = np.random.randint(0, self.learning_set.shape[1])
      predict = self.forward(self.learning_set[tmp_i][tmp_j])
      if self.answers[5*tmp_i + tmp_j] - predict != 0:
        self.weights[1:] += self.learning_rate * (self.answers[5*tmp_i + tmp_j] - predict) * self.learning_set[tmp_i][tmp_j]
        self.weights[0] += self.learning_rate * (self.answers[5*tmp_i + tmp_j] - predict)
        self.current_lifetime = 0
      else:
        self.current_lifetime += 1
        current_predict_number = self.current_predict_number()
        if(self.current_lifetime > self.best_lifetime & current_predict_number > self.best_predict_number):
          self.best_predict_number = current_predict_number
          self.best_lifetime = self.current_lifetime
          if (self.best_weights != self.weights).all():
            self.best_weights = np.copy(self.weights)

  def forward(self, data):
    dot = np.dot(data, self.weights[1:]) + self.weights[0]
    return self.activ_func(dot)

  def forward_predict(self, data):
    dot = np.dot(data, self.best_weights[1:]) + self.best_weights[0]
    return self.activ_func(dot)

  def activ_func(self, dot):
    return 1 if dot >= 0 else -1

  def predict(self, data):
    return self.forward_predict(data)

  def current_predict_number(self):
    correct_number = 0
    for i in range(10):
      for j in range(5):
        if self.forward(self.learning_set[i][j] == self.answers[5*i + j]):
          correct_number += 1
    return correct_number

# Using perceptrons

Data preparation

In [5]:
text_data = np.empty([10, 35, 5], dtype='float32')
for i in range(10):
  text_data[i] = np.loadtxt('digits/' + str(i) + '.txt')

In [6]:
numbers_separated = np.empty([10, 5, 7, 5], dtype='float32')
for i in range(10):
  numbers_separated[i] = np.array_split(text_data[i], 5)

In [7]:
numbers = np.empty([10, 5, 35], dtype='float32')
for i in range(10):
  for j in range(5):
    numbers[i][j] = numbers_separated[i][j].flatten()

In [8]:
E = np.copy(numbers)

Matrix of correct answers

In [9]:
T = np.full((10, 10 * 5), -1, dtype='int')
for i in range(10):
  T[i][i*5 : i*5 + 5] = 1

Create perceptrons

In [10]:
SPLAs = []
PLAs = []
PLARs = []
for i in range(10):
  SPLAs.append(SPLA(35, E, T[i]))
  PLAs.append(PLA(35, E, T[i]))
  PLARs.append(PLAR(35, E, T[i]))

Learning

In [11]:
for i in range(10):
  SPLAs[i].train(2500)
  PLAs[i].train(2500)
  PLARs[i].train(2500)

Checking the operation of perceptrons on training data

In [12]:
SPLAs_predict = np.zeros(10)
PLAs_predict = np.zeros(10)
PLARs_predict = np.zeros(10)
for i in range(10):
  for digit in range(10):
    for example in range(5):
      if(SPLAs[i].predict(E[digit][example]) == T[i][digit*5 + example]):
        SPLAs_predict[i] += 1
      else:
        print(f'SPLA {i} {digit} {example}')
      if(PLAs[i].predict(E[digit][example]) == T[i][digit*5 + example]):
        PLAs_predict[i] += 1
      else:
        print(f'PLA {i} {digit} {example}')
      if(PLARs[i].predict(E[digit][example]) == T[i][digit*5 + example]):
        PLARs_predict[i] += 1
      else:
        print(f'PLAR {i} {digit} {example}')

Learning outcomes

In [13]:
SPLAs_predict

array([50., 50., 50., 50., 50., 50., 50., 50., 50., 50.])

In [14]:
PLAs_predict

array([50., 50., 50., 50., 50., 50., 50., 50., 50., 50.])

In [15]:
PLARs_predict

array([50., 50., 50., 50., 50., 50., 50., 50., 50., 50.])

In [16]:
f'{round(np.sum(SPLAs_predict) / 500 * 100, 1)} % correct answers to training data for SLPA.'

'100.0 % correct answers to training data for SLPA.'

In [17]:
f'{round(np.sum(PLAs_predict) / 500 * 100, 1)} % correct answers to training data for PLA.'

'100.0 % correct answers to training data for PLA.'

In [18]:
f'{round(np.sum(PLARs_predict) / 500 * 100, 1)} % correct answers to training data for PLAR.'

'100.0 % correct answers to training data for PLAR.'